<a href="https://colab.research.google.com/github/aenoboa1/ML_EmotionAPP/blob/main/LSTM_FAR_DAISEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1ZUOa5Oh880SqaTlsIf3yhtY1SXl8H2ru&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=1ZUOa5Oh880SqaTlsIf3yhtY1SXl8H2ru
To: /content/Results_Experiment_FER_DAiSEE.zip
100% 6.79G/6.79G [01:28<00:00, 76.7MB/s]


In [ ]:
! unzip -qq /content/Results_Experiment_FER_DAiSEE.zip -d /content/DAISEE_FER

In [ ]:
!git clone https://github.com/f-imp/Engagement-Classification-exploiting-Facial-Emotion-s-Temporal-Sequences

Cloning into 'Engagement-Classification-exploiting-Facial-Emotion-s-Temporal-Sequences'...
remote: Enumerating objects: 482, done.
remote: Counting objects: 100% (482/482), done.
remote: Compressing objects: 100% (386/386), done.
remote: Total 482 (delta 108), reused 457 (delta 92), pack-reused 0
Receiving objects: 100% (482/482), 33.80 MiB | 27.91 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [ ]:
import numpy



In [ ]:
!mv '/content/Engagement-Classification-exploiting-Facial-Emotion-s-Temporal-Sequences/Supports' '/usr/local/lib/python3.7/dist-packages/'

In [ ]:
!ls -

ls: cannot access '-': No such file or directory


In [ ]:
numpy.__file__

'/usr/local/lib/python3.7/dist-packages/numpy/__init__.py'

In [ ]:
from tensorflow.keras.optimizers import Adam


In [ ]:
import os
from Supports.ML.Core_ML_Experiment import execute_model_daisee
from Supports.ML.Models import model_1, model_2, model_3, model_4, model_5, model_6
import pandas as pd

In [ ]:
pathname_folder_results = "Datasets_Generated/"

folder_FER_results_window = "FER_Daisee_window"
folderpath_FER_results_window = os.path.join(pathname_folder_results, folder_FER_results_window)

In [ ]:
parent_folder = "FER_Daisee_Results"
os.makedirs(parent_folder, exist_ok=True)

In [ ]:
max_epochs = 300
bs_hyperset = [32, 64, 128, 256]
lr_hyperset = [0.001]
window_set = [3, 5, 7, 9]
binary_mode_set = [True, False]
# 0 undersample, 1 oversampling, -1 take original
tackle_unbalance_strategy = [-1, 0, 1]
name_models = ["LSTM_with_return_sequences", "Convolutional_Network", "LSTM",
               "LSTM_with_return_sequences2", "LSTM_with_return_sequences3", "LSTM_with_return_sequences4"]
models_set = [model_1, model_2, model_3, model_4, model_5, model_6]

In [ ]:
i = 1
for w in window_set:
    folder_exp = "FER_Daisee_Results_Window_"+str(w)
    pathfolder_exp = os.path.join(parent_folder, folder_exp)
    os.makedirs(pathfolder_exp, exist_ok=True)
    for binary_version in binary_mode_set:
      for each_strategy in tackle_unbalance_strategy:
        for each_bs in bs_hyperset:
          for each_lr in lr_hyperset:
            for (each_model, name) in zip(models_set,name_models):
              n_total = len(window_set)*len(binary_mode_set)*len(tackle_unbalance_strategy)*len(bs_hyperset)*len(lr_hyperset)*len(models_set)

              pathname = name+"_"
              pathname += "Window_"+str(w)+"_"

              problem_cardinality = ""
              if binary_version:
                problem_cardinality = "Binary"
              else:
                problem_cardinality = "Multiclass"

              pathname += problem_cardinality+"_"

              handle_unbalancing = ""
              if each_strategy == 1:
                handle_unbalancing = "Oversampling"
              elif each_strategy == 0:
                handle_unbalancing = "Downsampling"
              else:
                handle_unbalancing = "No_sampling"

              pathname += handle_unbalancing+"_"

              pathname += str(each_bs)+"_"

              lr_scientific = "{:.0e}".format(each_lr)
              pathname += lr_scientific



              folderpath = os.path.join(pathfolder_exp, pathname)
              if not os.path.exists(os.path.join(folderpath, "summary.csv")):
                local_data = {'pathfile':[], 'Model':[],
                              'Data_window':[],'Epochs':[],
                              'Batch_size':[],'Learning_rate':[],
                              'Binary':[],'Tackle_Unbalanced':[],
                              'Accuracy':[],'Loss':[]}
                os.makedirs(folderpath, exist_ok=True)
                pathfile_img_performance = os.path.join(folderpath, "performance.png")
                pathfile_csv = os.path.join(folderpath, "data_experiment.csv")
                pathfile_confusion_matrix = os.path.join(folderpath, "confusion_matrix")
                pathfile_model = os.path.join(folderpath, "model.h5")
                pathfile_classification_report = os.path.join(folderpath, "classification_report.csv")

                filepath_array_X = os.path.join(os.getcwd(),
                                                "Datasets_Generated",
                                                "FER_Daisee_window",
                                                "window_"+str(w),
                                                "All",
                                                "X_All_window_"+str(w)+".npy")

                filepath_array_Y = os.path.join(os.getcwd(),
                                                "Datasets_Generated",
                                                "FER_Daisee_window",
                                                "window_"+str(w),
                                                "All",
                                                "y_All_window_"+str(w)+".npy")

                print("\nExperiment # {}/{}\nFolder name:\t{}\n".format(str(i),str(n_total), pathname))
                accuracy, loss, epochs_before_early_stopping = execute_model_daisee(path_X = filepath_array_X,
                                                                                    path_y = filepath_array_Y,
                                                                                    model_nn = each_model,
                                                                                    n_epochs = max_epochs,
                                                                                    bs = each_bs,
                                                                                    lr = each_lr,
                                                                                    path_image_performances = pathfile_img_performance,
                                                                                    path_csv_experiment_data = pathfile_csv,
                                                                                    path_confmatrix = pathfile_confusion_matrix,
                                                                                    path_classification_report = pathfile_classification_report,
                                                                                    path_model = pathfile_model,
                                                                                    verbosity = False,
                                                                                    binary = binary_version,
                                                                                    sampling_strategy = each_strategy)

                local_data['pathfile'].append(os.path.join(parent_folder,pathname))
                local_data['Model'].append(name)
                local_data['Data_window'].append(w)
                local_data['Epochs'].append(epochs_before_early_stopping)
                local_data['Batch_size'].append(each_bs)
                local_data['Learning_rate'].append(each_lr)
                local_data['Binary'].append(problem_cardinality)
                local_data['Tackle_Unbalanced'].append(handle_unbalancing)
                local_data['Accuracy'].append(accuracy)
                local_data['Loss'].append(loss)
                pd.DataFrame(local_data).to_csv(os.path.join(folderpath, "summary.csv"))
                print("The experiment {} IS FINISHED!".format(pathname))
              else:
                print("Experiment {} already done!!".format(pathname))

              i += 1

Experiment LSTM_with_return_sequences_Window_3_Binary_No_sampling_32_1e-03 already done!!
Experiment Convolutional_Network_Window_3_Binary_No_sampling_32_1e-03 already done!!
Experiment LSTM_Window_3_Binary_No_sampling_32_1e-03 already done!!
Experiment LSTM_with_return_sequences2_Window_3_Binary_No_sampling_32_1e-03 already done!!
Experiment LSTM_with_return_sequences3_Window_3_Binary_No_sampling_32_1e-03 already done!!
Experiment LSTM_with_return_sequences4_Window_3_Binary_No_sampling_32_1e-03 already done!!
Experiment LSTM_with_return_sequences_Window_3_Binary_No_sampling_64_1e-03 already done!!
Experiment Convolutional_Network_Window_3_Binary_No_sampling_64_1e-03 already done!!
Experiment LSTM_Window_3_Binary_No_sampling_64_1e-03 already done!!
Experiment LSTM_with_return_sequences2_Window_3_Binary_No_sampling_64_1e-03 already done!!
Experiment LSTM_with_return_sequences3_Window_3_Binary_No_sampling_64_1e-03 already done!!
Experiment LSTM_with_return_sequences4_Window_3_Binary_No_s